Programmer: Giovanni Vecchione
Date: 2/12/24
Subject: Project 1

Create a Deep Neural Networks model using the data set of your choice. Model should have at least 4 hidden layers. Dropout and Batch Normalization layers are not counted.

In [ ]:
import tensorflow as tf
import matplotlib as mpl


